In [8]:
# To enable cost analysis of our sorting, we will have global counters
# for the number of swap operations performed, and the number of
# comparisons performed.
#
# Call resetCounts() before performing a sort
def resetCounts():
    global swapCount
    swapCount = 0
    global compareCount
    compareCount = 0
    
# Call reportCounts() after performing a sort to see what the cost was    
def reportCounts():
    global swapCount
    global compareCount
    return str(swapCount) + " swaps, "+ str(compareCount) + " compares"

# This function swaps the two items at locations i and j in array A.
# (it also logs the usage for later reporting)
def swap(A, i, j):
    # Uncomment to see details of swaps:
    #print("swapping " + str(A[i]) + " with " + str(A[j]))
    temp = A[i]
    A[i] = A[j]
    A[j] = temp
    global swapCount
    swapCount += 1

# This function compares the two items at locations i and j in array A.
# It returns true if A[i] is greater than A[j] (the items must be
# comparable with the > operator).
# (The function also logs the usage for later reporting)
def compare(A, i, j):
    # Uncomment to see details of comparisons:
    #print("comparing " + str(A[i]) + " with " + str(A[j]))
    global compareCount
    compareCount += 1
    return A[i] > A[j]

In [20]:
import math
import random

# Sort the items from i to j (inclusive) in A
def quickSort(A, i, j, method):
    # If there are fewer than 2 items to sort, then the items are
    # de-facto sorted
    if j<=i: return

    # Select the pivot item
    pivotIndex = 0
    if method == "a":
        # Use item i as the pivot
        pivotIndex = i;
    elif method == "b":
        # Use the midpoint as the pivot
        pivotIndex = i + math.floor((j-i)/2)
    elif method == "c":
        # Use a randomly selected item as the pivot
        pivotIndex = random.randint(i,j)
    
    if pivotIndex != i: swap(A, i, pivotIndex)
    
    # Partition the items from i+1 to j into two sets:
    # items less than or equal to the pivot, and
    # items greater than the pivot.
    
    # During processing, the array will look like this:
    # 1: | pivot | <=pivot || unprocessed | >pivot |
    # or this:
    # 2: | pivot | <=pivot | g | unprocessed || >pivot |
    # or this:
    # 3: | pivot | <=pivot | g | unprocessed | l | >pivot |
    #
    # Where pivot is the single pivot,
    # <=pivot is the items that have been found to be <= the pivot,
    # g is a single item that has been found to be greater than the pivot,
    # unprocessed is items that have not yet been compared to the pivot,
    # l is a single item that has been found to be less than the pivot,
    # >pivot is the items that have been found to be > the pivot
    #
    # We start in state 1 (with <=pivot and >pivot empty), and advance by
    # moving the || separator through the unprocessed items from the left
    # until we encounter an item that is > than the pivot.
    # We then move to state 2, and advance by moving the || separator
    # through the unprocessed items from the right until we encounter
    # an item that is <= the pivot.
    # We are then in state 3.  We swap g and l so they are both in the
    # correct partitions, and return to state 1.
    #
    # (We stop when unprocessed is empty.)

    # Keep track of the left and right ends of the unprocessed region
    left = i+1
    right = j
    
    while left <= right:
        # We're in state 1.  Advance through the unprocessed region from the
        # left until we find an item > than pivot
        while left <= right and not(compare(A, left, i)):
            left += 1
        # left is now the index of g (an item that is > pivot).
        # (Or left > right, meaning that we have finished the partition - 
        # either way, left is one spot to the right of the last <=pivot item
        # found so far).
            
        # We're in state 2.  Advance through the unprocessed region from the
        # right until we find an item <= pivot
        while left < right and compare(A, right, i):
            right -= 1
        # If left is equal to right, then we have finished the partition).
        if left == right: right -= 1
        # right is now the index of l (or right is < left - either way,
        # right is one spot to the left of the first >pivot item found so far).
        
        # We're in state 3. (Or we're done)
        if left < right:
            swap(A, left, right)
            left += 1
            right -= 1
        
        # Go back to state 1
        
    # Partition complete. left is the index above the last item that is <= pivot.
    # Put the pivot in between the <=pivot and >pivot.
    if left != i+1: swap(A, i, left-1)
    
    # Uncomment to print the intermediate results
    #print(A)
    
    # The pivot is now in its final sorted position. So it's time to recurse!
    # Sort everything to the left of the pivot.
    quickSort(A, i, left-2, method)
    # Sort everything to the right of the pivot.
    quickSort(A, left, j, method)

In [23]:
# Here's a test array (see https://www.youtube.com/watch?v=lyZQPjUT5B4)
A = [3,0,1,8,7,2,5,4,6,9]

In [24]:
# Run quickSort on A

resetCounts()
print("starting:")
print(A)
print("")

quickSort(A, 0, len(A)-1, "c")

print("sorted:")
print(A)
reportCounts()

starting:
[3, 0, 1, 8, 7, 2, 5, 4, 6, 9]

sorted:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


'9 swaps, 24 compares'